In [11]:
#!pip install keras
import keras
import numpy as np
from keras.datasets import cifar10, cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, ZeroPadding2D, Dense, Dropout, Activation, Add, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D
from keras.initializers import glorot_uniform
from keras import optimizers
from keras.layers.normalization import BatchNormalization

In [2]:
from keras import backend as K
if('tensorflow' == K.backend()):
    import tensorflow as tf
    from keras.backend.tensorflow_backend import set_session
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)

In [3]:
def res_block(X, f, filters, stage, block, increase = False, s = 2):
    
    # Defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters   # Retrieving filters
    X_shortcut = X         # Saving input value
    stride = (1,1)
    
    if increase:
        stride = (s,s)
        
        #Shortcut Path
        X_shortcut = Conv2D(F3, (1,1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed = 0))(X_shortcut)
        X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = stride, padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters  = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)
    
    return X

In [4]:
def ResNet50(input_shape, classes):
    
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0), padding = 'same')(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = res_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1, increase = True)
    X = res_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = res_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3 (≈4 lines)
    X = res_block(X, f = 3, filters = [128,128,512], stage = 3, block = 'a', s = 2, increase = True)
    X = res_block(X, 3, [128,128,512], stage = 3, block = 'b')
    X = res_block(X, 3, [128,128,512], stage = 3, block = 'c')
    X = res_block(X, 3, [128,128,512], stage = 3, block = 'd')
    
    # Stage 4 (≈6 lines)
    X = res_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = 'a', s = 2, increase = True)
    X = res_block(X, 3, [256, 256, 1024], stage = 4, block = 'b')
    X = res_block(X, 3, [256, 256, 1024], stage = 4, block = 'c')
    X = res_block(X, 3, [256, 256, 1024], stage = 4, block = 'd')
    X = res_block(X, 3, [256, 256, 1024], stage = 4, block = 'e')
    X = res_block(X, 3, [256, 256, 1024], stage = 4, block = 'f')
    
    # Stage 5 (≈3 lines)
    X = res_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block = 'a', s = 2, increase = True)
    X = res_block(X, 3, [512, 512, 2048], stage = 5, block = 'b')
    X = res_block(X, 3, [512, 512, 2048], stage = 5, block = 'c')
    
    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D((2,2))(X)
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='output', kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    resnet = Model(inputs = X_input, outputs = X, name='ResNet50')

    return resnet

## CIFAR-10

In [5]:
# Load data
(c10_x_train, c10_y_train), (c10_x_test, c10_y_test) = cifar10.load_data()

# Hyper-parameters
c10_epochs = 25
c10_batch_size = 128
c10_num_classes = 10
c10_input_shape = c10_x_train.shape[1:]

# Pre-processing
c10_x_train = c10_x_train.astype('float32')
c10_x_test = c10_x_test.astype('float32')
c10_x_train = c10_x_train/255
c10_x_test = c10_x_test/255
c10_y_train = keras.utils.to_categorical(c10_y_train, c10_num_classes)
c10_y_test = keras.utils.to_categorical(c10_y_test, c10_num_classes)

In [6]:
c10_resnet = ResNet50(c10_input_shape, c10_num_classes)
#resnet.summary()

In [7]:
c10_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
c10_resnet.fit(c10_x_train, c10_y_train, epochs = c10_epochs, batch_size = c10_batch_size)

Epoch 1/25
50000/50000 [==============================] - 94s 2ms/step - loss: 1.8825 - accuracy: 0.3776
Epoch 2/25
50000/50000 [==============================] - 62s 1ms/step - loss: 1.5095 - accuracy: 0.4841
Epoch 3/25
50000/50000 [==============================] - 62s 1ms/step - loss: 1.4118 - accuracy: 0.5159
Epoch 4/25
50000/50000 [==============================] - 61s 1ms/step - loss: 1.4042 - accuracy: 0.5123
Epoch 5/25
50000/50000 [==============================] - 62s 1ms/step - loss: 1.1764 - accuracy: 0.5958
Epoch 6/25
50000/50000 [==============================] - 62s 1ms/step - loss: 1.2968 - accuracy: 0.5472
Epoch 7/25
50000/50000 [==============================] - 61s 1ms/step - loss: 1.0824 - accuracy: 0.6258
Epoch 8/25
50000/50000 [==============================] - 62s 1ms/step - loss: 0.8963 - accuracy: 0.6850
Epoch 9/25
50000/50000 [==============================] - 62s 1ms/step - loss: 0.7642 - accuracy: 0.7328
Epoch 10/25
50000/50000 [==============================

In [8]:
scores = c10_resnet.evaluate(c10_x_test, c10_y_test)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 10s 964us/step
Test loss: 1.603287338256836
Test accuracy: 0.6518999934196472


In [9]:
c10_resnet.save('c10_resnet50.h5')
c10_resnet.save_weights('c10_resnet50_weights.h5')

## CIFAR-100

In [12]:
# Load data
(c100_x_train, c100_y_train), (c100_x_test, c100_y_test) = cifar100.load_data()

# Hyper-parameters
c100_epochs = 50
c100_batch_size = 128
c100_num_classes = 100
c100_input_shape = c100_x_train.shape[1:]

# Pre-processing
c100_x_train = c100_x_train.astype('float32')
c100_x_test = c100_x_test.astype('float32')
c100_x_train = c100_x_train/255
c100_x_test = c100_x_test/255
c100_y_train = keras.utils.to_categorical(c100_y_train, c100_num_classes)
c100_y_test = keras.utils.to_categorical(c100_y_test, c100_num_classes)

In [15]:
c100_resnet = ResNet50(c100_input_shape, c100_num_classes)
#resnet.summary()

In [16]:
c100_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
c100_resnet.fit(c100_x_train, c100_y_train, epochs = c100_epochs, batch_size = c100_batch_size)

Epoch 1/50
50000/50000 [==============================] - 89s 2ms/step - loss: 4.3600 - accuracy: 0.0858
Epoch 2/50
50000/50000 [==============================] - 61s 1ms/step - loss: 3.5302 - accuracy: 0.1735
Epoch 3/50
50000/50000 [==============================] - 61s 1ms/step - loss: 3.1701 - accuracy: 0.2293
Epoch 4/50
50000/50000 [==============================] - 61s 1ms/step - loss: 3.0435 - accuracy: 0.2556
Epoch 5/50
50000/50000 [==============================] - 61s 1ms/step - loss: 2.7241 - accuracy: 0.3151
Epoch 6/50
50000/50000 [==============================] - 61s 1ms/step - loss: 2.4980 - accuracy: 0.3649
Epoch 7/50
50000/50000 [==============================] - 61s 1ms/step - loss: 2.2493 - accuracy: 0.4137
Epoch 8/50
50000/50000 [==============================] - 61s 1ms/step - loss: 2.0092 - accuracy: 0.4691
Epoch 9/50
50000/50000 [==============================] - 61s 1ms/step - loss: 1.8249 - accuracy: 0.5074
Epoch 10/50
50000/50000 [==============================

In [17]:
scores = c100_resnet.evaluate(c100_x_test, c100_y_test)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 9s 888us/step
Test loss: 5.504680083465576
Test accuracy: 0.40220001339912415


In [18]:
c100_resnet.save('c100_resnet50.h5')
c100_resnet.save_weights('c100_resnet50_weights.h5')